# Demo of the Data Query with LangChain

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

from dotenv import load_dotenv
import sqlite3
load_dotenv()

True

In [7]:
# create a connection to the SQLite database
conn = sqlite3.connect('../data/uvb.db')

# query the database using SQL
query = "SELECT * \
        FROM erythemal INNER JOIN locations \
        ON erythemal.loc_id = locations.loc_id \
        WHERE erythemal.loc_id IN ('CO11', 'NZ01') \
        ORDER BY RANDOM() \
        LIMIT 5"

# print the result
result = pd.read_sql_query(query, conn)
print(result)

# close the database connection
conn.close()

  loc_id  qc     azi    elev    zen   decl   amass  soldst  erythemal   
0   CO11   0  210.21  3220.0  20.81  21.88  1.0693  1.0166   0.261240  \
1   CO11   0  198.28  3220.0  64.96 -22.37  2.3530  0.9855   0.036057   
2   CO11   0  207.01  3220.0  65.12 -20.04  2.3669  0.9840   0.019909   
3   NZ01   0   49.20   390.0  67.79   9.24  2.6307  1.0029   0.030049   
4   NZ01   0  342.69   390.0  70.09  23.21  2.9149  1.0166   0.014459   

         date             time  time_sec  time_diff_sec  intgr_erythemal   
0  2018-07-12  12:57:00.000000     46620           60.0         15.67440  \
1  2020-12-04  13:09:00.000000     47340           60.0          2.16342   
2  2019-01-20  14:02:00.000000     50520           60.0          1.19454   
3  2006-04-14  09:42:00.000000     34920          180.0          5.40882   
4  2020-06-29  13:57:00.000000     50220          180.0          2.60262   

                      loc_name       lat         lon  
0  Colorado,Steamboat Springs,  40.45509 -106.744

In [8]:
db = SQLDatabase.from_uri("sqlite:///../data/uvb.db")
llm = OpenAI(temperature=0)
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
db_chain.run("Which location has seen the highest total erythemal irradiance in March 2022?")



> Entering new SQLDatabaseChain chain...
Which location has seen the highest total erythemal irradiance in March 2022? 
SQLQuery: SELECT loc_name, SUM(intgr_erythemal) AS total_erythemal FROM erythemal JOIN locations ON erythemal.loc_id = locations.loc_id WHERE date BETWEEN '2022-03-01' AND '2022-03-31' GROUP BY loc_name ORDER BY total_erythemal DESC LIMIT 5;
SQLResult: [('Hawaii,Waimea,Mauna Loa', 181407.4488557997), ('Louisiana,Baton Rouge', 98985.01551299995), ('New Mexico,Las Cruces', 97343.29793880021), ('Florida,Homestead', 90216.18464400046), ('Texas,Seguin', 89317.40853059999)]
Answer: The location with the highest total erythemal irradiance in March 2022 is Hawaii,Waimea,Mauna Loa.
> Finished chain.


' The location with the highest total erythemal irradiance in March 2022 is Hawaii,Waimea,Mauna Loa.'

In [9]:
db_chain.run("Across all locations in Colorado, what was the sunniest day in March 2022?")



> Entering new SQLDatabaseChain chain...
Across all locations in Colorado, what was the sunniest day in March 2022? 
SQLQuery: SELECT date, MAX(intgr_erythemal) FROM erythemal INNER JOIN locations ON erythemal.loc_id = locations.loc_id WHERE locations.loc_name LIKE '%Colorado%' AND date BETWEEN '2022-03-01' AND '2022-03-31' GROUP BY date ORDER BY MAX(intgr_erythemal) DESC LIMIT 5;
SQLResult: [('2022-03-16', 415.33884), ('2022-03-07', 106.2909), ('2022-03-11', 34.662384), ('2022-03-26', 34.3242), ('2022-03-28', 31.024800000000003)]
Answer: The sunniest day in March 2022 across all locations in Colorado was March 16th.
> Finished chain.


' The sunniest day in March 2022 across all locations in Colorado was March 16th.'

In [10]:
db_chain.run("Which location to the west of Colorado has the heighest elevation?")



> Entering new SQLDatabaseChain chain...
Which location to the west of Colorado has the heighest elevation? 
SQLQuery: SELECT loc_name, lat, lon, elev FROM locations WHERE lon < -106.744491 ORDER BY elev DESC LIMIT 5;
SQLResult: [('Hawaii,Waimea,Mauna Loa', 19.536161, -155.576261, 3409.0), ('Arizona,Flagstaff', 36.059, -112.184, 2073.0), ('Utah,Logan', 41.667, -111.892, 1369.0), ('Washington,Pullman', 46.76, -117.192, 805.0), ('Alaska,Fairbanks', 65.118467, -147.432315, 509.0)]
Answer: The location to the west of Colorado with the highest elevation is Hawaii,Waimea,Mauna Loa with an elevation of 3409.0.
> Finished chain.


' The location to the west of Colorado with the highest elevation is Hawaii,Waimea,Mauna Loa with an elevation of 3409.0.'